In [4]:
import lvpyio as lv
import numpy as np
import matplotlib.pyplot as plt
data_dir = '/home/eigil/DTU/particle_tracking_02456/data/Deep Learning Turbulence Project/ML_project_data/ImgPreproc(no_subtract)/'


In [37]:
import torch
from torch.utils.data import Dataset, DataLoader
import glob
from torchvision import transforms

In [40]:
class CustomDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(glob.glob(self.root + '/B*.im7')) - 4 # index by middle image, remove 2 images from start and end

    def __getitem__(self, idx):
        # Generate your data here
        # For example, let's generate a random tensor as data
        
        image_series = torch.zeros(5, 616, 2048)

        # Load images
        for i in range(0, 5):
            img = lv.read_buffer(data_dir + '/B' + str(idx + i + 1).zfill(5) + '.im7')
            img = torch.from_numpy(img[0].as_masked_array().data) # Currently hardcoded to camera 1

            image_series[i, :, :] = img

        if self.transform:
            image_series = self.transform(image_series)

        # For simplicity, let's assume all samples have the same label (0 in this case)
        #label = 0

        return image_series


In [41]:
dataset = CustomDataset(data_dir)

In [47]:
data = dataset.__getitem__(95)

In [57]:
upper_lower = (100, 300)

cutout_transform = transforms.Lambda(lambda x: x[:, upper_lower[0]:upper_lower[1], :]) 

# Define a transform if needed
transform = transforms.Compose([
    cutout_transform
])

dataset = CustomDataset(data_dir, transform=transform)
# Create an instance of your custom dataset
#custom_dataset = CustomDataset(num_samples=1000, transform=transform)

# Create a DataLoader for your custom dataset
custom_dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Now you can iterate over the dataloader to get batches of generated data
#for batch in custom_dataloader:
#    generated_data, labels = batch
#    # Your training/validation/testing logic goes here


In [50]:
for batch in custom_dataloader:
    break

In [51]:
batch.shape

torch.Size([2, 5, 200, 2048])

In [ ]:
def load_frame(frame_number, camera_number=0):
    buffer = lv.read_buffer(data_dir + f"B{str(frame_number+1).zfill(5)}.im7")
    image = buffer[camera_number].as_masked_array().data
    return image

In [48]:
data.shape

torch.Size([5, 200, 2048])

In [61]:
import os
os.chdir('/home/eigil/DTU/particle_tracking_02456/')
print(os.getcwd())
import src.data.simulate_dataset

/home/eigil/DTU/particle_tracking_02456
